# bla

# pip install 'lamindb[bionty,jupyter]' pronto
# cellxgene-schema has pinned dependencies. Therefore we recommend installing it into a separate environment using `uv` or `pipx`
# uv tool install cellxgene-schema==5.3.2

!lamin init --storage ./test-cellxgene-curate --modules bionty

In [ ]:
import lamindb as ln
import bionty as bt

ln.settings.verbosity = "error"

In [ ]:
adata = ln.core.datasets.small_dataset3_cellxgene(with_obs_defaults=True)
adata.write_h5ad("small_cxg.h5ad")
adata

In [ ]:
!MPLBACKEND=agg uvx cellxgene-schema validate small_cxg.h5ad

In [ ]:
ln.examples.cellxgene.save_cxg_defaults()

In [ ]:
adata.obs

In [ ]:
standardization_map = {
    "organism": (bt.Organism, "organism_ontology_term_id"),
    "assay": (bt.ExperimentalFactor, "assay_ontology_term_id"),
    "tissue": (bt.Tissue, "tissue_ontology_term_id"),
    "self_reported_ethnicity": (
        bt.Ethnicity,
        "self_reported_ethnicity_ontology_term_id",
    ),
    "cell_type": (bt.CellType, "cell_type_ontology_term_id"),
}

for col, (bt_class, new_col) in standardization_map.items():
    adata.obs[new_col] = bt_class.standardize(
        adata.obs[col], field="name", return_field="ontology_id"
    )

adata.obs = adata.obs.drop(columns=list(standardization_map.keys()))

In [ ]:
schema = ln.examples.cellxgene.get_cxg_schema("5.3.0", field_types="ontology_id")